In [1]:
import numpy as np
import random
import pandas as pd
from tqdm import tqdm
import random

In [2]:
pd.set_option('display.max_colwidth', None)

### 1. Draw method ###

In [3]:
#method to set up initial conditions
#types:1,2,3,4
def draw():
    pack = [(1,1),(2,1),(3,1),(4,1),(5,1),(6,1),(7,1),(8,1),(9,1),(10,1),(11,1),(12,1),(13,1)] + [(1,2),(2,2),(3,2),(4,2),(5,2),(6,2),(7,2),(8,2),(9,2),(10,2),(11,2),(12,2),(13,2)] +  [(1,3),(2,3),(3,3),(4,3),(5,3),(6,3),(7,3),(8,3),(9,3),(10,3),(11,3),(12,3),(13,3)] +  [(1,4),(2,4),(3,4),(4,4),(5,4),(6,4),(7,4),(8,4),(9,4),(10,4),(11,4),(12,4),(13,4)]
    pack.sort()
    #antes = [25,50,100,200]
    player_card = []
    community_card = []
    for i in range(2):
        p = random.choice(pack)
        pack.remove(p)
        player_card.append(p)
    for i in range(3):
        c = random.choice(pack)
        pack.remove(c)
        community_card.append(c)
    ante = 100
    return player_card, community_card, ante

In [4]:
draw()

([(12, 2), (7, 4)], [(6, 2), (4, 3), (2, 4)], 100)

### Output table for draw ###

In [5]:
df1 = pd.DataFrame(columns = ['draw output'])
for i in range(5):
    df1 = df1.append({'draw output': draw()}, ignore_index = True)

In [6]:
df1

,draw output
0,"([(11, 3), (10, 1)], [(2, 1), (2, 4), (10, 4)], 100)"
1,"([(10, 2), (5, 4)], [(6, 2), (2, 3), (3, 1)], 100)"
2,"([(7, 1), (5, 2)], [(1, 2), (6, 3), (5, 3)], 100)"
3,"([(7, 2), (3, 1)], [(9, 1), (11, 3), (5, 2)], 100)"
4,"([(3, 3), (13, 1)], [(6, 4), (9, 3), (2, 3)], 100)"


This verifies that my draw() is working properly.

### 2. Step method ###

In [7]:
#method takes as an input a state s and an action, and returns a sample of the next state s’ and reward r
def step(state, action,com_card):
    #state: ([turned over cards], ante)
    cards = state[0]
    ante = state[1]
    if action == 0 or action == 'fold':
        return ('terminal', (0-ante))
    if action == 1 or action == "bet1":
        ante = ante
        com = com_card[0]
        com_card = com_card[1:]
        next_cards = cards + [com]
        next_state = (next_cards,ante)
        reward = 0
    if action == 2 or action == "bet2":
        ante = ante*2
        com = com_card[0]
        com_card = com_card[1:]
        next_cards = cards + [com]
        next_state = (next_cards,ante)
        reward = 0
    if action == 3 or action == "bet3":
        ante = ante*3
        com = com_card[0]
        com_card = com_card[1:]
        next_cards = cards + [com]
        next_state = (next_cards,ante)
        reward = 0
    if len(next_cards) == 5:
        reward = pay_table(next_cards,ante)
        return ('terminal',reward)
    return (next_state,reward)
    
        
        

In [8]:
def pay_table(cards,ante):
    value = [cards[i][0] for i in range(len(cards))]
    color = [cards[i][1] for i in range(len(cards))]
    value.sort()
    color.sort()
    same_color = True
    for i in range(0,len(color)-1):
        if color[i] != color [i+1]:
            same_color = False
    # Royal Flush
    if value == [1,10,11,12,13] and same_color == True:
        return 500*ante
    consecutive = True
    for i in range(0,len(color)-1):
        if value[i+1] != value[i]+1:
            consecutive = False
    #Straight Flush
    if consecutive == True and same_color == True:
        return 100*ante
    
    #Four of a kind
    if True in [value.count(val) == 4 for val in value]:
        return 40*ante
    #Full House
    if True in [value.count(val) == 3 for val in value] and True in [value.count(val) == 2 for val in value]:
        return 10*ante
    #Flush
    if same_color == True:
        return 6*ante
    #Straight
    if consecutive == True:
        return 4*ante
    three_a_kind = False
    #Three of a Kind
    if True in [value.count(val) == 3 for val in value]:
        return 3*ante
    pairs = [i for i in value if value.count(i)==2]
    #Two Pairs
    if len(pairs)==4:
           return 2*ante
    #Pair of Jacks or Better
    if True in [i in pairs for i in [11,12,13,1]]:
        return 1*ante
    #Pair of 6s thre 10s
    if True in [i in pairs for i in [6,7,8,9,10]]:
        return 0
    #otherwise
    return (0-ante)
    
    
    
        
    

In [9]:
test_input = [[([(11, 4), (12, 3),(6, 2), (10, 1)],100),'bet1', [ (7, 2)]],[([(11, 4), (12, 4),(13, 4), (1, 4)],100),'bet2', [(10, 4)]],[([(11, 2), (8, 3),(6, 2)],100),'bet2', [(10, 3), (7, 2)]],[([(11, 4), (10, 4),(9, 4), (8, 4)],100),'bet3', [ (7, 4)]],[([(3, 4), (3, 3),(3, 2), (10, 1)],100),'bet2', [ (3, 2)]],[([(1, 4), (3, 3),(6, 2), (10, 1)],100),'bet1', [ (1, 2)]]]

In [10]:
for i in test_input:
    print(i)
    print(step(i[0],i[1],i[2]))

[([(11, 4), (12, 3), (6, 2), (10, 1)], 100), 'bet1', [(7, 2)]]
('terminal', -100)
[([(11, 4), (12, 4), (13, 4), (1, 4)], 100), 'bet2', [(10, 4)]]
('terminal', 100000)
[([(11, 2), (8, 3), (6, 2)], 100), 'bet2', [(10, 3), (7, 2)]]
(([(11, 2), (8, 3), (6, 2), (10, 3)], 200), 0)
[([(11, 4), (10, 4), (9, 4), (8, 4)], 100), 'bet3', [(7, 4)]]
('terminal', 30000)
[([(3, 4), (3, 3), (3, 2), (10, 1)], 100), 'bet2', [(3, 2)]]
('terminal', 8000)
[([(1, 4), (3, 3), (6, 2), (10, 1)], 100), 'bet1', [(1, 2)]]
('terminal', 100)


In [11]:
expected = [('terminal', -100),('terminal', 100000),(([(11, 2), (8, 3), (6, 2), (10, 3)], 200), 0),('terminal', 30000),('terminal', 8000),('terminal', 100)]
df2 = pd.DataFrame(columns = ['input','actual_output','expected_output'])
for i in range(len(test_input)):
    result = step(test_input[i][0],test_input[i][1],test_input[i][2])
    df2 = df2.append({'input':test_input[i], 'actual_output':result,'expected_output':expected[i]}, ignore_index = True)
    

### Input and output table for step() ###

In [12]:
df2

,input,actual_output,expected_output
0,"[([(11, 4), (12, 3), (6, 2), (10, 1)], 100), bet1, [(7, 2)]]","(terminal, -100)","(terminal, -100)"
1,"[([(11, 4), (12, 4), (13, 4), (1, 4)], 100), bet2, [(10, 4)]]","(terminal, 100000)","(terminal, 100000)"
2,"[([(11, 2), (8, 3), (6, 2)], 100), bet2, [(10, 3), (7, 2)]]","(([(11, 2), (8, 3), (6, 2), (10, 3)], 200), 0)","(([(11, 2), (8, 3), (6, 2), (10, 3)], 200), 0)"
3,"[([(11, 4), (10, 4), (9, 4), (8, 4)], 100), bet3, [(7, 4)]]","(terminal, 30000)","(terminal, 30000)"
4,"[([(3, 4), (3, 3), (3, 2), (10, 1)], 100), bet2, [(3, 2)]]","(terminal, 8000)","(terminal, 8000)"
5,"[([(1, 4), (3, 3), (6, 2), (10, 1)], 100), bet1, [(1, 2)]]","(terminal, 100)","(terminal, 100)"


This verifies that my environment is working properly.

In [13]:
#helper method to implement epsilon-greedy strategy
def ep_greedy(epsilon,Q,state):
    # fold, bet1, bet2, bet3
    actions = [0,1,2,3]
    prob = np.random.random()
    #nongreedy method:
    if prob < epsilon:
        A = random.choice(actions)
    else:
        [a,b,c,d] = find_index(state,Q)
        #now index contains all turned-over cards information
        ante_ind = int(state[1]/100)-1
        
        
        action_index = np.argmax(Q[a,b,c,d,ante_ind])
        A = actions[action_index]
    return A

In [14]:
#helper method to find index for input state
def find_index (state,Q):
    turned_card = state[0]
    num_card = len(state[0])
    index = []
    for i in turned_card:
        val = i[0]
        typ = i[1]
        card_index = 13*(typ - 1) + val
        index.append(card_index)

      
    #if there are non-flipped cards:
    if len(index)<4:
        left = 4-len(index)
        for i in range(left):
            index.append(0)
    return index

### 3. Q_learning algorithm ###

In [16]:
def Q_learning(alpha,epsilon,gamma,episode_num,max_steps):
    q_values = np.zeros([13*4+1, 13*4+1, 13*4+1, 13*4+1, 3*3,4],dtype = np.float32)
    for i in range(episode_num):
        #initialize state
        player_card, com_card, ante = draw()
        state = (player_card, ante)
        for i in range(max_steps):
            # Select action based on epsilon greedy
            action = ep_greedy(epsilon,q_values,state)
            #take action, observe s,r
            (next_state,reward) = step(state, action,com_card)
            com_card = com_card[1:]
            [a1,b1,c1,d1] = find_index(state,q_values)
            #now index contains all turned-over cards information
            ante_ind = int(state[1]/100)-1
            if next_state != 'terminal':
                #find index for next state
                [a2,b2,c2,d2] = find_index(next_state,q_values)
                next_ante_ind = int(next_state[1]/100)-1                
                q_values[a1,b1,c1,d1,ante_ind,action] += alpha*(reward + gamma*np.max(q_values[a2,b2,c2,d2,next_ante_ind]) - q_values[a1,b1,c1,d1,ante_ind,action]) 
                #print(alpha*(reward + gamma*np.max(q_values[a2,b2,c2,d2,next_ante_ind]) - q_values[a1,b1,c1,d1,ante_ind,action]) )
                state = next_state
            else:
                q_values[a1,b1,c1,d1,ante_ind,action] += alpha*(reward + gamma*0 - q_values[a1,b1,c1,d1,ante_ind,action])
                break
    return q_values


In [17]:
Q_learn = Q_learning(0.01,0.1,0.1,1000000,10)

53

 Visualize the optimal action for Q learning 

In [19]:
result_qlearn1 = [] #positive r
result_qlearn2 = [] #negative r
for i in range(53):
  for j in range(53):
    for k in range(53):
      for h in range(53):
        for g in range(9): #ante
          #for f in range(4): #action
          action = np.argmax(Q_learn[i][j][k][h][g])
          r = Q_learn[i][j][k][h][g][action]
          if r > 0:
            result_qlearn1.append({"card1":i,"card2":j,"card3":k,"card4":h,"ante":g,"action":action,"reward":r})
          elif r < 0:
            result_qlearn2.append({"card1":i,"card2":j,"card3":k,"card4":h,"ante":g,"action":action,"reward":r})
            



In [43]:
#randomly choose 10 state-action pairs from the positive case and negative case respectively
positive1 = random.sample(result_qlearn1,10)
negative1 = random.sample(result_qlearn2,10)
negative1


[{'action': 0,
  'ante': 0,
  'card1': 24,
  'card2': 42,
  'card3': 8,
  'card4': 19,
  'reward': -1.0},
 {'action': 0,
  'ante': 0,
  'card1': 52,
  'card2': 18,
  'card3': 30,
  'card4': 2,
  'reward': -1.0},
 {'action': 0,
  'ante': 0,
  'card1': 41,
  'card2': 28,
  'card3': 37,
  'card4': 1,
  'reward': -1.0},
 {'action': 0,
  'ante': 0,
  'card1': 15,
  'card2': 28,
  'card3': 52,
  'card4': 43,
  'reward': -1.0},
 {'action': 0,
  'ante': 0,
  'card1': 46,
  'card2': 50,
  'card3': 41,
  'card4': 26,
  'reward': -1.0},
 {'action': 0,
  'ante': 0,
  'card1': 15,
  'card2': 37,
  'card3': 46,
  'card4': 8,
  'reward': -1.0},
 {'action': 0,
  'ante': 0,
  'card1': 6,
  'card2': 8,
  'card3': 39,
  'card4': 12,
  'reward': -1.0},
 {'action': 0,
  'ante': 0,
  'card1': 14,
  'card2': 4,
  'card3': 32,
  'card4': 28,
  'reward': -1.0},
 {'action': 0,
  'ante': 0,
  'card1': 36,
  'card2': 7,
  'card3': 3,
  'card4': 47,
  'reward': -1.0},
 {'action': 0,
  'ante': 0,
  'card1': 7,
  'c

In [44]:
df3 = pd.DataFrame(columns = ['card1','card2','card3','card4','ante','action','value'])
for i in positive1:
    df3 = df3.append({'card1': i['card1'],'card2':i['card2'],'card3':i['card3'],'card4':i['card4'],'ante':i['ante'],'action':i['action'],'value':i['reward']}, ignore_index = True)
for i in negative1:
    df3 = df3.append({'card1': i['card1'],'card2':i['card2'],'card3':i['card3'],'card4':i['card4'],'ante':i['ante'],'action':i['action'],'value':i['reward']}, ignore_index = True)
df3
    

,card1,card2,card3,card4,ante,action,value
0,33.0,24.0,11.0,18.0,0.0,1.0,1.000000
1,16.0,37.0,12.0,38.0,0.0,2.0,2.000000
2,46.0,9.0,11.0,25.0,0.0,3.0,3.000000
3,22.0,39.0,11.0,28.0,0.0,1.0,1.000000
4,14.0,26.0,21.0,11.0,0.0,1.0,1.000000
5,18.0,49.0,10.0,0.0,0.0,1.0,0.003728
6,51.0,41.0,25.0,0.0,0.0,1.0,0.002000
7,14.0,26.0,13.0,12.0,0.0,1.0,1.000000
8,40.0,8.0,29.0,33.0,0.0,1.0,1.000000
9,41.0,1.0,25.0,27.0,0.0,2.0,2.000000


### 4. Monte Carlo ES Algorithm ### 

In [20]:
def MCES(episode_nums,gamma):
  actions = [0,1,2,3]
  pi = np.zeros([13*4+1, 13*4+1, 13*4+1, 13*4+1, 3*3],dtype = np.float32)
  q_values = np.zeros([13*4+1, 13*4+1, 13*4+1, 13*4+1, 3*3,4],dtype = np.float32)
  Return = np.zeros([13*4+1, 13*4+1, 13*4+1, 13*4+1, 3*3,4],dtype = np.float32)
  count_r = np.zeros([13*4+1, 13*4+1, 13*4+1, 13*4+1, 3*3,4],dtype = np.float32)
  for i in range(episode_nums):
    #randomly initialize state and action
    player_card, com_card, ante = draw()
    state = (player_card, ante)
    action = random.choice(actions)
    episode = []
    while state != 'terminal':
      (next_state,reward) = step(state, action,com_card)
      com_card = com_card[1:]
      episode.append([state,action,reward])
      action = random.choice(actions)
      state = next_state
    G = 0
    #set up prestate to record visited states
    prestate = []
    for i in range(len(episode)):
      prestate.append((episode[i][0],episode[i][1]))
    #loop through the episodes:
    for t in range(len(episode)-1,-1,-1):#backward
      epi = episode[t]
      G = gamma*G + epi[2]
      State = epi[0]
      Action = epi[1]
      #update q_values and pi
      if (((State,Action) in prestate[:t]) == False):
        [a1,b1,c1,d1] = find_index(State,q_values)
        ante_id = int(State[1]/100)-1
        Return[a1,b1,c1,d1,ante_id,Action] += G
        count_r[a1,b1,c1,d1,ante_id,Action] += 1
        q_values[a1,b1,c1,d1,ante_id,Action] = Return[a1,b1,c1,d1,ante_id,Action]/count_r[a1,b1,c1,d1,ante_id,Action]

        pi[a1,b1,c1,d1,ante_id] = np.argmax(q_values[a1,b1,c1,d1,ante_id])
       
  return q_values,pi




In [21]:
Q_mc,pi = MCES(1000000,0.1)

Visualize optimal action for Monte Carlo ES algorithm

In [22]:
result_qmc1 = [] #positive r
result_qmc2 = [] #negative r
for i in range(53):
  for j in range(53):
    for k in range(53):
      for h in range(53):
        for g in range(9): #ante
          #for f in range(4): #action
          action = np.argmax(Q_mc[i][j][k][h][g])
          r = Q_mc[i][j][k][h][g][action]
          if r > 0:
            result_qmc1.append({"card1":i,"card2":j,"card3":k,"card4":h,"ante":g,"action":action,"reward":r})
          elif r < 0:
            result_qmc2.append({"card1":i,"card2":j,"card3":k,"card4":h,"ante":g,"action":action,"reward":r})

In [41]:
#randomly choose 10 state-action pairs from the positive case and negative case respectively
positive2 = random.sample(result_qmc1,10)
negative2 = random.sample(result_qmc2,10)

In [42]:
df4 = pd.DataFrame(columns = ['card1','card2','card3','card4','ante','action','value'])
for i in positive2:
    df4 = df4.append({'card1': i['card1'],'card2':i['card2'],'card3':i['card3'],'card4':i['card4'],'ante':i['ante'],'action':i['action'],'value':i['reward']}, ignore_index = True)
for i in negative2:
    df4 = df4.append({'card1': i['card1'],'card2':i['card2'],'card3':i['card3'],'card4':i['card4'],'ante':i['ante'],'action':i['action'],'value':i['reward']}, ignore_index = True)
df4
  

,card1,card2,card3,card4,ante,action,value
0,38.0,18.0,31.0,25.0,5.0,3.0,3600.000000
1,12.0,50.0,18.0,44.0,5.0,1.0,1800.000000
2,3.0,38.0,31.0,0.0,0.0,3.0,60.000000
3,30.0,8.0,47.0,43.0,8.0,1.0,1800.000000
4,51.0,47.0,40.0,0.0,1.0,1.0,40.000000
5,2.0,11.0,32.0,1.0,1.0,2.0,400.000000
6,49.0,28.0,36.0,0.0,1.0,2.0,26.666666
7,4.0,38.0,14.0,0.0,0.0,1.0,10.000000
8,40.0,14.0,20.0,26.0,2.0,2.0,600.000000
9,37.0,50.0,36.0,33.0,5.0,1.0,600.000000


### 5. Calculate the MSE of these 2 algorithms

In [23]:
#helper method: compute MSE
def MSE(Q_learn,Q_mc):
    MSE = 0
    n = 53*53*53*53*9*4
    for i in range(53):
      for j in range(53):
        for k in range(53):
          for h in range(53):
            for g in range(9):
              for f in range(4):
                MSE += (Q_learn[i][j][k][h][g][f]-Q_mc[i][j][k][h][g][f])**2
    return MSE/n

In [24]:
MSE1 = MSE(Q_learn,Q_mc)
print("mean-squared error is", MSE1)

mean-squared error is 3168.4217590205258


In [ ]:
np.set_printoptions(threshold=np.inf)dd